# Bubble and Quick sort

## Common

In [25]:
import random
import operator
import timeit


idx = operator.itemgetter(0)
val = operator.itemgetter(1)


def pairs(arr):
    return zip(enumerate(arr), enumerate(arr[1:], 1))


def is_bad(pair):
    a, b = pair
    return val(a) > val(b)


def bad_pairs(arr):
    return filter(is_bad, pairs(arr))


def is_sorted(arr):
    return not any(bad_pairs(arr))


def move(arr, at, to):
    arr.insert(to, arr.pop(at))
    

def test_array():
    return [random.randint(0, 100) for _ in range(25)]


def time(sorting_function):
    return timeit.timeit(f'{sorting_function.__name__}(test_array())', number=2000, globals=globals())


def print_sorting(sorting_function):
    print(f'{sorting_function.__name__} ({time(sorting_function)} sec)')
    a = test_array()
    print(a)
    assert not is_sorted(a)
    sorting_function(a)
    assert is_sorted(a)
    print(a)

## Bubble sort

In [26]:
def bubble_sort(arr):
    while not is_sorted(arr):
        for a, b in bad_pairs(arr):
            move(arr, idx(a), idx(b))

## Quick sort

In [27]:
def fo(arr, i, p):
    if arr[i] > arr[p]:
        move(arr, i, p)
        p -= 1
    else:
        i += 1
    return i, p


def foo(arr, i, p):
    while i < p:
        i, p = fo(arr, i, p)
    return p


def quick_sort(arr, low=0, high=-1):
    current = low
    pivot = high if high >= 0 else high + len(arr)
    if current < pivot:
        pivot = foo(arr, current, pivot)
        quick_sort(arr, low, pivot - 1)
        quick_sort(arr, pivot + 1, high)

## Testing

In [28]:
print_sorting(bubble_sort)
print()
print_sorting(quick_sort)

bubble_sort (1.2692080870001519 sec)
[10, 16, 95, 28, 50, 45, 13, 84, 74, 52, 31, 42, 84, 78, 10, 5, 64, 74, 97, 23, 18, 13, 66, 50, 51]
[5, 10, 10, 13, 13, 16, 18, 23, 28, 31, 42, 45, 50, 50, 51, 52, 64, 66, 74, 74, 78, 84, 84, 95, 97]

quick_sort (0.5241844729998775 sec)
[68, 55, 77, 16, 42, 2, 4, 50, 77, 28, 36, 72, 84, 17, 59, 93, 33, 95, 86, 17, 28, 30, 91, 47, 44]
[2, 4, 16, 17, 17, 28, 28, 30, 33, 36, 42, 44, 47, 50, 55, 59, 68, 72, 77, 77, 84, 86, 91, 93, 95]
